In [ ]:
import os
import random
import time
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random
import time
import spacy
import numpy as np
from torch import Tensor

In [ ]:
TEXT = data.Field(tokenize = 'spacy', include_lengths=True)
LABEL = data.LabelField(dtype = torch.float) 

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print('Number of training examples: {}'.format(len(train_data)))
print('Number of validation examples: {}'.format(len(valid_data)))
print('Number of testing examples: {}'.format(len(test_data)))

In [ ]:
print(vars(train_data.examples[0]))
print(' '.join(vars(train_data.examples[0])['text']))

In [ ]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")

In [ ]:
!nvidia-smi

In [ ]:
TEXT.build_vocab(train_data, vectors = "glove.6B.100d")

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_                 
                 )
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
LABEL.vocab.itos

In [ ]:
TEXT.vocab.itos[:10] 

In [ ]:
word_dict = TEXT.vocab.stoi
print(len(word_dict))
print(word_dict['<unk>'], word_dict['<pad>'])
print(list(word_dict)[-1], word_dict[list(word_dict)[-1]])

In [ ]:
print(TEXT.vocab.vectors.shape)
print(TEXT.vocab.vectors)

In [ ]:
BATCH_SIZE = 32   

train_iterator = data.Iterator(
    train_data, 
    batch_size = BATCH_SIZE,
    device = device)

for batch in train_iterator:
    break
print(batch.text)
print(len(batch.text[0]))
print(batch.label)

In [ ]:
train_iterator = data.BucketIterator(
    train_data, 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

for batch in train_iterator:
    break
print(batch.text)
print(len(batch.text[0]))
print(batch.label)

In [ ]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_sizes = (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    sort_within_batch = True,
    device = device)

In [ ]:
class Model(nn.Module): 
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()

        self.hidden_him = hidden_dim
        self.n_layers = n_layers
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout)

        if self.bidirectional:
            self.fc = nn.Linear(hidden_dim * 2, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        output, (hidden, cell) = self.lstm(embedded)

        if self.bidirectional:
             hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else :
             hidden = self.dropout(hidden[-1,:,:])

        return self.fc(hidden.squeeze(0))


In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
#BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = Model(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)  


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)  # Count number of elements of all parameters

print('The model has {:,} trainable parameters'.format(count_parameters(model)))

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
print(type(pretrained_embeddings))
model.embedding.weight.data.copy_(pretrained_embeddings);

In [ ]:
optimizer = optim.Adam(model.parameters())   

In [ ]:
criterion = nn.BCEWithLogitsLoss()  

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad() 
                
        predictions = model(batch.text[0]).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()  

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            predictions = model(batch.text[0]).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn-model.pt')
    
    print('Epoch: {:02}'.format(epoch+1))
    print('\tTrain Loss: {:.3f} | Train Acc: {:.2f}%'.format(train_loss, train_acc*100))
    print('\t Val. Loss: {:.3f} |  Val. Acc: {:.2f}%'.format(valid_loss, valid_acc*100))

In [ ]:
model.load_state_dict(torch.load('rnn-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print('Test Loss: {:.3f} | Test Acc: {:.2f}%'.format(test_loss, test_acc*100))

Test Loss: 0.340 | Test Acc: 85.27%


In [ ]:
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "I do not like parties") 

0.2775626480579376

In [ ]:
predict_sentiment(model, " I like computer games") 

0.7981942892074585